In [1]:
!nvidia-smi

Thu May 16 08:04:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [3]:
!git clone https://github.com/WongKinYiu/yolov9.git

Cloning into 'yolov9'...
remote: Enumerating objects: 668, done.
remote: Counting objects: 100% (668/668), done.
remote: Compressing objects: 100% (405/405), done.
remote: Total 668 (delta 271), reused 575 (delta 246), pack-reused 0
Receiving objects: 100% (668/668), 3.23 MiB | 8.27 MiB/s, done.
Resolving deltas: 100% (271/271), done.


In [5]:
%cd yolov9
!pip3 install -r requirements.txt -q

[Errno 2] No such file or directory: 'yolov9'
/kaggle/working/yolov9


In [6]:
!pip3 install -q roboflow

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
jupyterlab 4.1.6 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.9.2 requires shapely>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.2.1 which is i

In [7]:
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [9]:
!ls -la {HOME}/weights

total 402440
drwxr-xr-x 2 root root      4096 May 16 08:06 .
drwxr-xr-x 5 root root      4096 May 16 08:06 ..
-rw-r--r-- 1 root root  51508261 Feb 18 12:36 gelan-c.pt
-rw-r--r-- 1 root root 117203713 Feb 18 12:36 gelan-e.pt
-rw-r--r-- 1 root root 103153312 Feb 18 12:36 yolov9-c.pt
-rw-r--r-- 1 root root 140217688 Feb 18 12:36 yolov9-e.pt


In [10]:
%cd {HOME}/yolov9

/kaggle/working/yolov9


In [11]:
from roboflow import Roboflow
rf = Roboflow(api_key="1gBpoxVirzj0NHBuskuN")
project = rf.workspace("capstone-design-2").project("fire-smoke-7rocs")
version = project.version(1)
dataset = version.download("yolov9")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to fire-smoke-1 in yolov9:: 100%|██████████| 1947/1947 [00:00<00:00, 8650.39it/s]


In [16]:
%cd {HOME}/yolov9

!python train.py \
--batch 32 --epochs 100 --img 640 --device 0,1 --min-items 0 --close-mosaic 15 \
--data {dataset.location}/data.yaml \
--weights {HOME}/weights/gelan-c.pt \
--cfg models/detect/gelan-c.yaml \
--hyp hyp.scratch-high.yaml

/kaggle/working/yolov9
2024-05-16 08:11:23.520654: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 08:11:23.520710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 08:11:23.522162: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=/kaggle/working/weights/gelan-c.pt, cfg=models/detect/gelan-c.yaml, data=/kaggle/working/yolov9/fire-smoke-1/data.yaml, hyp=hyp.s

In [24]:
!python export.py --weights /kaggle/working/yolov9/runs/train/exp/weights/best_striped.pt --include onnx

export: data=data/coco.yaml, weights=['/kaggle/working/yolov9/runs/train/exp/weights/best_striped.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLO 🚀 v0.1-89-g93f1a28 Python-3.10.13 torch-2.1.2 CPU

Fusing layers... 
gelan-c summary: 387 layers, 25229401 parameters, 0 gradients, 101.8 GFLOPs

PyTorch: starting from /kaggle/working/yolov9/runs/train/exp/weights/best_striped.pt with output shape (1, 7, 8400) (49.1 MB)

ONNX: starting export with onnx 1.16.0...
ONNX: export success ✅ 3.5s, saved as /kaggle/working/yolov9/runs/train/exp/weights/best_striped.onnx (96.5 MB)

Export complete (6.0s)
Results saved to /kaggle/working/yolov9/runs/train/exp/weights
Detect:          python detect.py --weights /kaggle/working/yolov9/runs/train/exp

In [25]:
!tar -czvf yolov9-b32-e100-img640.tar.gz /kaggle/working/yolov9/runs/train/exp

tar: Removing leading `/' from member names
/kaggle/working/yolov9/runs/train/exp/
/kaggle/working/yolov9/runs/train/exp/val_batch1_labels.jpg
/kaggle/working/yolov9/runs/train/exp/opt.yaml
/kaggle/working/yolov9/runs/train/exp/train_batch0.jpg
/kaggle/working/yolov9/runs/train/exp/hyp.yaml
/kaggle/working/yolov9/runs/train/exp/val_batch0_labels.jpg
/kaggle/working/yolov9/runs/train/exp/val_batch2_labels.jpg
/kaggle/working/yolov9/runs/train/exp/R_curve.png
/kaggle/working/yolov9/runs/train/exp/val_batch1_pred.jpg
/kaggle/working/yolov9/runs/train/exp/val_batch0_pred.jpg
/kaggle/working/yolov9/runs/train/exp/labels.jpg
/kaggle/working/yolov9/runs/train/exp/results.csv
/kaggle/working/yolov9/runs/train/exp/train_batch2.jpg
/kaggle/working/yolov9/runs/train/exp/labels_correlogram.jpg
/kaggle/working/yolov9/runs/train/exp/F1_curve.png
/kaggle/working/yolov9/runs/train/exp/events.out.tfevents.1715847117.3e13265f22a6.306.0
/kaggle/working/yolov9/runs/train/exp/P_curve.png
/kaggle/working/yo

In [ ]:
!ls {HOME}/yolov9/runs/train/exp/

In [ ]:
from IPython.display import Image

Image(filename=f"{HOME}/yolov9/runs/train/exp/results.png", width=1000)

In [ ]:
from IPython.display import Image

Image(filename=f"{HOME}/yolov9/runs/train/exp/confusion_matrix.png", width=1000)

In [ ]:
from IPython.display import Image

Image(filename=f"{HOME}/yolov9/runs/train/exp/val_batch0_pred.jpg", width=1000)

In [ ]:
%cd {HOME}/yolov9

!python val.py \
--img 640 --batch 32 --conf 0.001 --iou 0.7 --device 0 \
--data {dataset.location}/data.yaml \
--weights {HOME}/yolov9/runs/train/exp/weights/best.pt

In [ ]:
!python detect.py \
--img 1280 --conf 0.1 --device 0 \
--weights {HOME}/yolov9/runs/train/exp/weights/best.pt \
--source {dataset.location}/test/images

In [ ]:
import glob

from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/yolov9/runs/detect/exp3/*.jpg')[:2]:
      display(Image(filename=image_path, width=600))